In [2]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/label_for_dev109_test.csv"
#device = "103"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/label_for_dev109_test_prepared.csv"

df = pd.read_csv(csv_path)
#for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_score', 'iso_risk', 'lstm_score', 'lstm_risk', 'confidence', 'rf_risk', 'devID']:
for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'devID']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

#def clean_and_format_timestamp(val):
#    if pd.isnull(val):
#        return None
#    val = str(val).replace('(ICT)', '').strip()
#    try:
#        dt = pd.to_datetime(val)
#        return dt.isoformat()
#    except Exception:
#        return None  
#
#df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

#df = df.drop(columns=['Unnamed: 15', 'confidence','rf_risk'], errors='ignore')
#df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
#df = df[df['lng'] > 15.0]
df = df[df['soil'] > 0]
df = df[df['rain'] >= 0]
df = df[df['humi'] > 0]
#df = df[(df['devID'] == 104) | (df['devID'] == 103)]
df = df[(df['geo'] >= 0) & (df['geo'] < 40)]
df = df[df['temp'] > 0]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
#df = df[df['iso_score'] > -85000]
#df = df[df['lstm_score'] >= 0]
#df = df[df['timestamp'] > '3035-09-11']

#df = df[df['devID'] == int(device)]
df

,timestamp,device,devID,temp,humi,soil,rain,geo,lat,long,risk,ISO Risk,LSTM Risk,RF Risk,label


In [3]:
df.describe()

,timestamp,device,devID,temp,humi,soil,rain,geo,lat,ISO Risk,LSTM Risk,RF Risk
count,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_risk', 'iso_score', 'lstm_risk']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,device,devID,temp,humi,soil,rain,geo,lat,long,risk,ISO Risk,LSTM Risk,RF Risk,label


In [6]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')ผ
#df['hour'] = df.index.hourผ
#df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

SyntaxError: invalid syntax (2175007259.py, line 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    0 non-null      datetime64[ns]
 1   devID        0 non-null      float64       
 2   temp         0 non-null      float64       
 3   humi         0 non-null      float64       
 4   soil         0 non-null      float64       
 5   rain         0 non-null      float64       
 6   geo          0 non-null      float64       
 7   rain_ma_1h   0 non-null      float64       
 8   rain_ma_6h   0 non-null      float64       
 9   rain_ma_12h  0 non-null      float64       
 10  rain_ma_24h  0 non-null      float64       
 11  soil_ma_4h   0 non-null      float64       
 12  soil_ma_8h   0 non-null      float64       
 13  soil_ma_12h  0 non-null      float64       
 14  soil_ma_16h  0 non-null      float64       
 15  geo_ma_2min  0 non-null      float64       
 16  geo_ma_4min  0 non-n

In [ ]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [ ]:
#df = df.dropna(subset=['humi']).reset_index(drop=True)

In [ ]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [ ]:
#df = df.dropna()
#
#nan_counts = df.isna().sum()
#nan_columns = nan_counts[nan_counts > 0]
#print("df102")
#print("📌 คอลัมน์ที่มีค่า NaN:")
#print(nan_columns)

In [ ]:
#df[(df["iso_risk"]==2) & (df["lstm_risk"]==2)]

In [ ]:
df.describe()

,timestamp,devID,temp,humi,soil,rain,geo,rain_ma_1h,rain_ma_6h,rain_ma_12h,rain_ma_24h,soil_ma_4h,soil_ma_8h,soil_ma_12h,soil_ma_16h,geo_ma_2min,geo_ma_4min,geo_ma_6min,geo_ma_8min
count,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['devID'].value_counts()

Series([], Name: count, dtype: int64)

In [ ]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
